In [1]:
!pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=94faa70ac46e349aefa39757c7e8b36970b46f013084fd6a6c7991f9bd4206a3
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [2]:
!pip install shap

In [3]:
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
from google.colab import drive
import pandas as pd
import yfinance as yf
from datetime import datetime
# Mount Google Drive
drive.mount('/content/drive')

# Load ticker symbols from Symbols.csv
file_path = '/content/drive/Shareddrives/capstone project-stock market liquidity/Symbols.csv'  # Update with your file path
try:
    symbols_df = pd.read_csv(file_path)
    print(symbols_df.head())
    tickers = symbols_df['0'].tolist()  # Ensure 'Symbol' is the correct column name
except Exception as e:
    print("Error reading Symbols.csv:", e)
    tickers = []

print("Tickers loaded:", tickers)

# Initialize a list for failed downloads
failed_tickers = []

# Download data while skipping missing tickers
valid_tickers = []
data_frames = []

for ticker in tickers:
    try:
        print(f"Downloading data for: {ticker}")
        data = yf.download(ticker, start="2020-01-01", end="2024-12-25", progress=False)
        if not data.empty:
            data.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
            data['Ticker'] = ticker  # Add a column to identify the ticker
            data_frames.append(data)
            valid_tickers.append(ticker)
        else:
            failed_tickers.append(ticker)
    except Exception as e:
        print(f"Failed to download data for {ticker}: {e}")
        failed_tickers.append(ticker)

# Concatenate data for all valid tickers
if data_frames:
    combined_data = pd.concat(data_frames)
    print("Combined data shape:", combined_data.shape)
else:
    print("No valid tickers downloaded.")

# Print tickers that failed
if failed_tickers:
    print("\nThe following tickers could not be downloaded:")
    print(failed_tickers)
else:
    print("\nAll tickers were successfully downloaded.")

# Flatten the multi-level columns
data=combined_data.copy()
print(data.head(5))
# Create custom features: High-Low difference and Open-Close difference
grouped_data = {}

# RSI Calculation
def calculate_rsi(prices, period=14):
    prices = pd.Series(prices).astype(float).dropna()
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

# MACD Calculation
def calculate_macd(prices, fast=12, slow=26):
    prices = pd.Series(prices).astype(float).dropna()
    exp1 = prices.ewm(span=fast, adjust=False).mean()
    exp2 = prices.ewm(span=slow, adjust=False).mean()
    return exp1 - exp2

# Perform calculations for each ticker
for ticker in tickers:
    # Extract data for the specific ticker
    ticker_data = data[data['Ticker']==ticker]

    # Calculate High-Low and Open-Close differences
    ticker_data['High_Low_Diff'] = ticker_data['High'] - ticker_data['Low']
    ticker_data['Open_Close_Diff'] = ticker_data['Open'] - ticker_data['Close']

    # Add Rolling Averages
    ticker_data['Adj_Close_5d_Rolling'] = ticker_data['Adj Close'].rolling(window=5).mean()
    ticker_data['Adj_Close_10d_Rolling'] = ticker_data['Adj Close'].rolling(window=10).mean()
    ticker_data['Adj_Close_20d_Rolling'] = ticker_data['Adj Close'].rolling(window=20).mean()
    ticker_data['Volatility'] = ticker_data['Adj Close'].rolling(window=10).std()
    # Add Binary Feature: Is Adj Close up (1) or down (0) compared to previous day
    ticker_data['Adj_Close_Up'] = (ticker_data['Adj Close'].diff() > 0).astype(int)
    ticker_data['RSI'] = calculate_rsi(ticker_data['Adj Close'])
    ticker_data['MACD'] = calculate_macd(ticker_data['Adj Close'])
    ticker_data['Returns'] = ticker_data['Adj Close'].pct_change()

    # Keep only the relevant columns
    grouped_data[ticker] = ticker_data[['Adj Close', 'Volume',
                                       'High_Low_Diff', 'Open_Close_Diff',
                                       'Adj_Close_5d_Rolling',
                                       'Adj_Close_10d_Rolling',
                                       'Adj_Close_20d_Rolling', 'Volatility', 'Adj_Close_Up', 'RSI', 'MACD', 'Returns']]

# Combine all tickers into a MultiIndex DataFrame
data_filtered = pd.concat(grouped_data, axis=1)





Mounted at /content/drive
   Unnamed: 0        0
0           0  ABRA.TA
1           1  ACCL.TA
2           2  ACKR.TA
3           3  ACRO.TA
4           4  ADGR.TA
Tickers loaded: ['ABRA.TA', 'ACCL.TA', 'ACKR.TA', 'ACRO.TA', 'ADGR.TA', 'ARDM.TA', 'AFHL.TA', 'AFPR.TA', 'AFRE.TA', 'AICS-M.TA', 'ARPT.TA', 'ARTS.TA', 'ALAR.TA', 'ALBA.TA', 'ALMD.TA', 'ALLT.TA', 'ALMY.TA', 'AMDA.TA', 'ALMA.TA', 'ALHE.TA', 'ALRPR.TA', 'ALUMA.TA', 'AMAN.TA', 'AMD.TA', 'AMRK.TA', 'AMOT.TA', 'AMRM.TA', 'ANGL.TA', 'APLP.TA', 'AQUA.TA', 'ARD.TA', 'ARAD.TA', 'ARAN.TA', 'ARBE.TA', 'ARGO.TA', 'ARIN.TA', 'ARYT.TA', 'ARF.TA', 'ASHO.TA', 'ASHG.TA', 'ASGR.TA', 'AUDC.TA', 'AUGN.TA', 'AURA.TA', 'AUSA-M.TA', 'AUIS.TA', 'AMX.TA', 'AVER.TA', 'AVGD.TA', 'AVGL.TA', 'AVIA.TA', 'AVIV.TA', 'AVRT.TA', 'AILN.TA', 'AYAL.TA', 'AZRM.TA', 'AZRG.TA', 'BCOM.TA', 'BKFR.TA', 'BVGG.TA', 'BRAN.TA', 'BRKT.TA', 'BVC.TA', 'ORL.TA', 'BHNY.TA', 'BSEN.TA', 'BEZQ.TA', 'BIG.TA', 'BIGT.TA', 'BKRY.TA', 'BIOV.TA', 'BOLT.TA', 'BLRX.TA', 'BIMT.TA', 'BIRM.

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['HDHA.TA']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


ERROR:yfinance:Could not get exchangeTimezoneName for ticker '???.TA' reason: 'chart'
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['???.TA']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['PMCN.TA']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['UNCR.TA']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['UNVO.TA']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


Combined data shape: (552035, 7)

The following tickers could not be downloaded:
['HDHA.TA', '???.TA', 'PMCN.TA', 'UNCR.TA', 'UNVO.TA']
             Adj Close       Close        High         Low        Open  \
Date                                                                     
2020-01-01  209.965805  209.965805  217.540894  203.944595  207.246597   
2020-01-02  218.123596  218.123596  220.454407  204.333099  213.753403   
2020-01-05  212.685104  212.685104  217.055298  210.742798  218.123596   
2020-01-06  202.099396  202.099396  212.685104  201.031097  212.685104   
2020-01-07  210.742798  210.742798  213.656296  202.099396  202.099396   

            Volume   Ticker  
Date                         
2020-01-01  311952  ABRA.TA  
2020-01-02  798593  ABRA.TA  
2020-01-05  280145  ABRA.TA  
2020-01-06  282834  ABRA.TA  
2020-01-07  168562  ABRA.TA  


Streaming output truncated to the last 5000 lines.
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_data['Adj_Close_Up'] = (ticker_data['Adj Close'].diff() > 0).astype(int)
<ipython-input-3-b9677de2d71c>:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_data['RSI'] = calculate_rsi(ticker_data['Adj Close'])
<ipython-input-3-b9677de2d71c>:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_data['MA

### Filling empty date

In [4]:
if '2024-04-07' in data_filtered.index:
    # Get the index position of '2024-04-07'
    idx = data_filtered.index.get_loc('2024-04-07')

    # Replace each column's value for '2024-04-07' with its next row's value
    for col in data_filtered.columns:
        if idx + 1 < len(data_filtered):  # Ensure the next row exists
            next_value = data_filtered.iloc[idx + 1][col]
            data_filtered.loc['2024-04-07', col] = next_value

In [5]:
data_filtered.columns

MultiIndex([('ABRA.TA',             'Adj Close'),
            ('ABRA.TA',                'Volume'),
            ('ABRA.TA',         'High_Low_Diff'),
            ('ABRA.TA',       'Open_Close_Diff'),
            ('ABRA.TA',  'Adj_Close_5d_Rolling'),
            ('ABRA.TA', 'Adj_Close_10d_Rolling'),
            ('ABRA.TA', 'Adj_Close_20d_Rolling'),
            ('ABRA.TA',            'Volatility'),
            ('ABRA.TA',          'Adj_Close_Up'),
            ('ABRA.TA',                   'RSI'),
            ...
            ( 'ZUR.TA',         'High_Low_Diff'),
            ( 'ZUR.TA',       'Open_Close_Diff'),
            ( 'ZUR.TA',  'Adj_Close_5d_Rolling'),
            ( 'ZUR.TA', 'Adj_Close_10d_Rolling'),
            ( 'ZUR.TA', 'Adj_Close_20d_Rolling'),
            ( 'ZUR.TA',            'Volatility'),
            ( 'ZUR.TA',          'Adj_Close_Up'),
            ( 'ZUR.TA',                   'RSI'),
            ( 'ZUR.TA',                  'MACD'),
            ( 'ZUR.TA',           

In [6]:
data_filtered.to_csv('data_filtered_from_yfinance.csv')

In [8]:
import pandas as pd
from statsmodels.tsa.stattools import grangercausalitytests

# Extract 'Adj Close' data for each ticker and align by time
adj_close_data = {ticker: grouped_data[ticker]['Adj Close'] for ticker in grouped_data.keys()}
aligned_data = pd.concat(adj_close_data, axis=1)

# Drop NaN values (rows with missing data for any ticker)
aligned_data = aligned_data.dropna()

# Perform Granger causality test for each pair of tickers
results = {}
max_lag = 5  # Set maximum number of lags to test

causal_pairs = []

print("Performing Granger causality tests...")

for ticker_x in aligned_data.columns:
    for ticker_y in aligned_data.columns:
        if ticker_x != ticker_y:  # Avoid testing causality on the same ticker
            try:
                test_result = grangercausalitytests(aligned_data[[ticker_y, ticker_x]], max_lag, verbose=False)
                p_values = [test_result[lag][0]['ssr_ftest'][1] for lag in range(1, max_lag + 1)]
                print(p_values)
                if any(p < 0.05 for p in p_values):  # Check if any lag shows significant causality
                    causal_pairs.append((ticker_x, ticker_y, p_values))
            except Exception as e:
                continue

# Save results to a DataFrame
causal_df = pd.DataFrame(causal_pairs, columns=["Cause", "Effect", "P-Values"])
causal_df.to_csv("causal_pairs.csv", index=False)

# Print the results
if not causal_df.empty:
    print("\nTickers implying causality:")
    print(causal_df)
else:
    print("\nNo causal relationships found.")



Performing Granger causality tests...


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose i


No causal relationships found.


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print resul

## only the stock data:

In [ ]:
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
# import matplotlib.pyplot as plt
# import numpy as np
# import pandas as pd
# import yfinance as yf
# from sklearn.preprocessing import MinMaxScaler, RobustScaler
# from scipy.optimize import minimize

# # Input parameters
# tickers = valid_tickers  # Add all tickers here
# num_days_to_predict = 30  # Forecast days

# # Initialize DataFrames to store results
# # Initialize DataFrames to store results
# actual_vs_pred_df = pd.DataFrame(columns=['Date', 'Ticker', 'Actual', 'Predicted'])
# forecast_df = pd.DataFrame(columns=['Date', 'Ticker', 'Forecast', 'Days_Ahead'])
# metrics_df = pd.DataFrame(columns=['Ticker', 'MSE', 'MAE','MAPE', 'R2'])

# lstm_better_df_path = '/content/drive/Shareddrives/capstone project-stock market liquidity/LSTM Results/lstm_better_df.csv'
# lstm_better_df = pd.read_csv(lstm_better_df_path)

# # Extract the tickers to process LSTM only for those
# tickers_to_process = lstm_better_df['Ticker'].unique()

# # Filter tickers based on this list
# tickers = [ticker for ticker in tickers if ticker in tickers_to_process]

# valid_tickers = []
# for ticker in tickers:
#     ticker_data = data_filtered.xs(ticker, axis=1, level=0)['Adj Close']  # Extract 'Adj Close' for the ticker
#     if ticker_data.max() >= 50:  # Check if the max value is >= 50
#         valid_tickers.append(ticker)

# # Update tickers list
# tickers = valid_tickers

# # Custom loss function to include volatility
# import tensorflow.keras.backend as K
# def mape(y_true, y_pred):
#     return K.mean(K.abs((y_true - y_pred) / y_true)) * 100

# # Define SMAPE
# def smape(y_true, y_pred):
#     denominator = K.abs(y_true) + K.abs(y_pred)
#     return K.mean(2 * K.abs(y_true - y_pred) / denominator) * 100
# def volatility_loss(y_true, y_pred):
#     mse = K.mean(K.square(y_true - y_pred))
#     diff_true = K.abs(y_true[1:] - y_true[:-1])
#     diff_pred = K.abs(y_pred[1:] - y_pred[:-1])
#     volatility_penalty = K.mean(K.square(diff_true - diff_pred))
#     direction_error = K.mean(K.square(K.sign(y_true[1:] - y_true[:-1]) - K.sign(y_pred[1:] - y_pred[:-1])))
#     return mse + 0.1 * volatility_penalty+0.1*direction_error+0.1*smape(y_true, y_pred)

# # Function to create LSTM model
# def create_lstm_model(input_shape):
#     model = Sequential()
#     model.add(Input(shape=input_shape))
#     model.add(LSTM(50, return_sequences=True))
#     model.add(Dropout(0.2))
#     model.add(LSTM(50, return_sequences=False))
#     model.add(Dropout(0.2))
#     model.add(Dense(25))
#     model.add(Dense(1))
#     model.compile(optimizer='adam', loss=volatility_loss)
#     return model

# # Function to train and forecast for each stock
# def process_stock(ticker, data_filtered):
#     global actual_vs_pred_df, forecast_df, metrics_df

#     print(f"Processing ticker: {ticker}")

#     # Extract data for the ticker
#     ticker_data = data_filtered.xs(ticker, axis=1, level=0)
#     ticker_data['Volatility'] = ticker_data['Adj Close'].rolling(window=10).std()
#     ticker_data['Returns'] = ticker_data['Adj Close'].pct_change(fill_method=None)
#     ticker_data = ticker_data.dropna()

#     # Features and target
#     X = ticker_data.drop('Adj Close', axis=1)
#     y = ticker_data['Adj Close']
#     dates = ticker_data.index

#     # Scaling
#     scaler_X, scaler_y = RobustScaler(), RobustScaler()
#     X_scaled = scaler_X.fit_transform(X)
#     y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))
#     X_scaled = X_scaled.reshape(X_scaled.shape[0], 1, X_scaled.shape[1])

#     # Split data
#     split_index = int(0.9 * len(X_scaled))
#     X_train, X_test = X_scaled[:split_index], X_scaled[split_index:]
#     y_train, y_test = y_scaled[:split_index], y_scaled[split_index:]
#     train_dates, test_dates = dates[:split_index], dates[split_index:]

#     # Build LSTM model
#     model = create_lstm_model(input_shape=(X_train.shape[1], X_train.shape[2]))
#     model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=0)

#     # Predict
#     y_pred = model.predict(X_test)
#     def compute_correction_factor(y_actual, y_pred, window_size=30):
#       correction_factors = []
#       for start in range(0, len(y_actual), window_size):
#           end = start + window_size
#           actual_slice = y_actual[start:end]
#           pred_slice = y_pred[start:end]
#           if len(actual_slice) > 0 and np.mean(pred_slice) > 0:
#               correction_factors.append(np.mean(actual_slice) / np.mean(pred_slice))
#           else:
#               correction_factors.append(1)  # Default to no correction if invalid
#       return correction_factors
#     def apply_correction(y_pred, correction_factors, window_size=30):
#       corrected_predictions = []
#       for i, factor in enumerate(correction_factors):
#           start = i * window_size
#           end = start + window_size
#           corrected_predictions.extend(y_pred[start:end] * factor)
#       return np.array(corrected_predictions)


#     y_pred_inverse = scaler_y.inverse_transform(y_pred)
#     y_test_inverse = scaler_y.inverse_transform(y_test)
#     if np.any(np.isnan(y_test_inverse)) or np.any(np.isnan(y_pred_inverse)):
#         print(f"NaN values detected for ticker: {ticker}")
#         # Remove NaN values
#         y_test_inverse = y_test_inverse[~np.isnan(y_test_inverse)]
#         y_pred_inverse = y_pred_inverse[~np.isnan(y_pred_inverse)]

#     # Ensure both arrays have the same length
#     if len(y_test_inverse) == 0 or len(y_pred_inverse) == 0:
#         print(f"No valid predictions for ticker: {ticker}. Skipping metric calculation.")
#         return

#     adjustment = y_test_inverse[0] - y_pred_inverse[0]

#     # Apply the adjustment to all predicted values
#     y_pred_inverse_adjusted = y_pred_inverse + adjustment

#     # Ensure both arrays have the same length after adjustment
#     if len(y_test_inverse) != len(y_pred_inverse_adjusted):
#         min_length = min(len(y_test_inverse), len(y_pred_inverse_adjusted))
#         y_test_inverse = y_test_inverse[:min_length]
#         y_pred_inverse_adjusted = y_pred_inverse_adjusted[:min_length]

#     # Calculate metrics with adjusted predictions
#     mse = mean_squared_error(y_test_inverse, y_pred_inverse_adjusted)
#     mae = mean_absolute_error(y_test_inverse, y_pred_inverse_adjusted)
#     mape = mean_absolute_percentage_error(y_test_inverse, y_pred_inverse_adjusted)
#     r2 = r2_score(y_test_inverse, y_pred_inverse_adjusted)

#     print(f"Ticker: {ticker}, MSE: {mse}, MAE: {mae}, MAPE: {mape}, R2: {r2}")

#     # Append metrics to DataFrame
#     metrics_df.loc[len(metrics_df)] = [ticker, mse, mae, mape, r2]

#     # Append actual vs adjusted predicted values
#     temp_actual_pred = pd.DataFrame({
#         'Date': test_dates,
#         'Ticker': ticker,
#         'Actual': y_test_inverse.flatten(),
#         'Predicted': y_pred_inverse_adjusted.flatten()
#     })
#     actual_vs_pred_df = pd.concat([actual_vs_pred_df, temp_actual_pred], ignore_index=True)


#     # Forecast next n days
#     last_X = X_scaled[-1]
#     future_predictions = []
#     for _ in range(num_days_to_predict):
#         future_pred = model.predict(last_X.reshape(1, 1, X_scaled.shape[2]))
#         noise = np.random.normal(0, 0.05, future_pred.shape)  # Add noise for volatility
#         future_pred_with_noise = future_pred + noise
#         future_predictions.append(future_pred_with_noise[0, 0])
#         last_X = np.roll(last_X, -1)
#         last_X[-1] = future_pred_with_noise

#     # Inverse scale forecast
#     future_predictions_inverse = scaler_y.inverse_transform(np.array(future_predictions).reshape(-1, 1))

#     # Append forecast to DataFrame
#     future_dates = pd.date_range(dates[-1] + pd.Timedelta(days=1), periods=num_days_to_predict)
#     temp_forecast = pd.DataFrame({
#         'Date': future_dates,
#         'Ticker': ticker,
#         'Forecast': future_predictions_inverse.flatten(),
#         'Days_Ahead': np.arange(1, num_days_to_predict + 1)
#     })
#     forecast_df = pd.concat([forecast_df, temp_forecast], ignore_index=True)

#     # Plot results
#     plt.figure(figsize=(14, 8))
#     plt.plot(train_dates, scaler_y.inverse_transform(y_train), color='green', label='Actual Adj Close (Train)')
#     plt.plot(test_dates, y_test_inverse, color='blue', label='Actual Adj Close (Test)')
#     plt.plot(test_dates, y_pred_inverse_adjusted, color='red', label='Predicted Adj Close (Test)')
#     plt.plot(temp_forecast['Date'], temp_forecast['Forecast'], color='orange', label='Forecasted Prices')

#     plt.title('Stock Prices: Actual, Predicted, and Forecasted')
#     plt.xlabel('Date')
#     plt.ylabel('Adj Close')
#     plt.legend()
#     plt.show()

# # Load data
# start_date = "2020-01-01"
# end_date = pd.Timestamp.today().strftime('%Y-%m-%d')
# data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')

# # Process each stock
# for ticker in tickers:
#     process_stock(ticker, data)

# # Save results
# actual_vs_pred_df.to_csv('actual_vs_pred.csv', index=False)
# forecast_df.to_csv('forecast.csv', index=False)
# metrics_df.to_csv('metrics.csv', index=False)

## stocks data correlation:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def calculate_ticker_correlations_with_valid_returns(data_filtered, cutoff_date="2024-02-01"):
    """
    Filters tickers with valid Returns data before the cutoff date and calculates correlations.

    Args:
        data_filtered (pd.DataFrame): MultiIndex DataFrame with tickers and their features.
        cutoff_date (str): Date to filter tickers that were active before this date.

    Returns:
        value_corr (pd.DataFrame): Correlation matrix for adjusted close prices.
        trend_corr (pd.DataFrame): Correlation matrix for Returns.
    """
    # Step 1: Extract Adjusted Close Prices
    adj_close = data_filtered.xs('Adj Close', axis=1, level=1)

    # Step 2: Extract Returns
    returns = data_filtered.xs('Returns', axis=1, level=1)

    # Step 3: Filter Tickers Active Before Cutoff Date
    active_tickers = []
    for ticker in returns.columns:
        # Check if the ticker has non-null Returns before the cutoff date
        if returns.loc[:cutoff_date, ticker].notnull().any():
            active_tickers.append(ticker)

    # Filter adjusted close and returns for active tickers
    adj_close = adj_close[active_tickers]
    returns = returns[active_tickers]

    # Step 4: Drop NaN Values Per Ticker
    returns = returns.apply(lambda x: x.dropna(), axis=0)

    # Step 5: Compute Correlation Matrices
    value_corr = adj_close.corr()  # Correlation of adjusted close prices
    trend_corr = returns.corr()   # Correlation of returns (trends)

    # Step 6: Visualize Correlations
    plt.figure(figsize=(12, 8))
    sns.heatmap(value_corr, annot=False, cmap="coolwarm", cbar=True)
    plt.title("Correlation Matrix: Adjusted Close Prices")
    plt.show()

    plt.figure(figsize=(12, 8))
    sns.heatmap(trend_corr, annot=False, cmap="coolwarm", cbar=True)
    plt.title("Correlation Matrix: Returns (Trends)")
    plt.show()

    return value_corr, trend_corr

# Recalculate correlations using filtered Returns
value_corr, trend_corr = calculate_ticker_correlations_with_valid_returns(data_filtered, cutoff_date="2024-02-01")


valid_tickers_in_corr = [ticker for ticker in valid_tickers if ticker in value_corr.index]
valid_tickers_in_corr_trend = [ticker for ticker in valid_tickers if ticker in trend_corr.index]

# Filter the correlation matrices based on the filtered valid tickers
value_corr = value_corr.loc[valid_tickers_in_corr, valid_tickers_in_corr]
trend_corr = trend_corr.loc[valid_tickers_in_corr_trend, valid_tickers_in_corr_trend]

def show_top_correlated_tickers(correlation_matrix, threshold=0.5, top_n=10):
    """
    Prints the top N tickers with the highest correlations for each ticker, above the given threshold.

    Args:
        correlation_matrix (pd.DataFrame): Correlation matrix (e.g., from value_corr or trend_corr).
        threshold (float): Minimum correlation threshold to consider.
        top_n (int): Number of top correlated tickers to display per ticker.
    """
    top_corr_tickers = {}
    for ticker in correlation_matrix.columns:
        # Filter tickers with correlation > threshold (excluding self-correlation)
        correlated_tickers = (
            correlation_matrix[ticker][correlation_matrix[ticker] > threshold]
            .drop(ticker)
            .sort_values(ascending=False)  # Sort by correlation value
        )
        if not correlated_tickers.empty:
            top_corr_tickers[ticker] = correlated_tickers.head(top_n).index.tolist()

    # Print results
    for ticker, correlated in top_corr_tickers.items():
        print(f"{ticker} is highly correlated with: {', '.join(correlated)}")

    return top_corr_tickers


# Call the function for value correlations
print("Top 10 Correlations for Adjusted Close Prices:")
top_corr_values = show_top_correlated_tickers(value_corr, threshold=0.5, top_n=10)

# Show top 10 correlations for trends
print("\nTop 10 Correlations for Trends (Percentage Changes):")
top_corr_trends = show_top_correlated_tickers(trend_corr, threshold=0.5, top_n=10)



Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import pandas as pd

def save_correlation_dict_to_csv(corr_dict, filename):
    """
    Converts a correlation dictionary to a DataFrame and saves it to a CSV file.

    Args:
        corr_dict (dict): Dictionary where keys are tickers and values are lists of correlated tickers.
        filename (str): Name of the CSV file to save.
    """
    # Convert the dictionary to a list of rows
    rows = []
    for key, values in corr_dict.items():
        for value in values:
            rows.append({'Ticker': key, 'Correlated_Ticker': value})

    # Convert rows to a DataFrame
    corr_df = pd.DataFrame(rows)

    # Save the DataFrame to a CSV file
    corr_df.to_csv(filename, index=False)
    print(f"Saved correlation dictionary to {filename}")

# Save `top_corr_values` and `top_corr_trends` to CSV
save_correlation_dict_to_csv(top_corr_values, "top_corr_values.csv")
save_correlation_dict_to_csv(top_corr_trends, "top_corr_trends.csv")


Saved correlation dictionary to top_corr_values.csv
Saved correlation dictionary to top_corr_trends.csv


## All data per stock:

In [ ]:
target_data = data_filtered['ACKR.TA'].dropna(subset=['Adj Close'])
target_data

,Adj Close,Volume,High_Low_Diff,Open_Close_Diff,Adj_Close_5d_Rolling,Adj_Close_10d_Rolling,Adj_Close_20d_Rolling,Volatility,Adj_Close_Up,RSI,MACD,Returns
Date,,,,,,,,,,,,
2021-06-08,389.084534,410232.0,29.000000,5.100006,NaN,NaN,NaN,NaN,0.0,NaN,0.000000,NaN
2021-06-09,401.904480,306082.0,19.299988,2.800018,NaN,NaN,NaN,NaN,1.0,NaN,1.022674,0.032949
2021-06-10,430.383026,282783.0,37.000000,-28.000000,NaN,NaN,NaN,NaN,1.0,NaN,4.084052,0.070859
2021-06-13,435.144684,283875.0,38.700012,-5.200012,NaN,NaN,NaN,NaN,1.0,NaN,6.815874,0.011064
2021-06-14,431.023987,85812.0,25.700012,17.299988,417.508142,NaN,NaN,NaN,0.0,NaN,8.549799,-0.009470
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-18,830.000000,96689.0,15.000000,-5.000000,819.840002,813.909998,771.654996,15.209900,1.0,75.148197,48.622527,0.006061
2024-12-19,834.500000,180757.0,33.399963,-8.200012,820.740002,818.059998,780.199997,14.516522,1.0,71.376054,47.635285,0.005422
2024-12-22,832.900024,140924.0,29.900024,2.099976,826.860010,821.640002,788.334998,13.120401,0.0,58.475214,46.191319,-0.001917


In [ ]:
target_data = data_filtered['ABRA.TA'].dropna(subset=['Adj Close'])
target_start_date = target_data.index.min()
print(target_start_date)
valid_stocks=[col[0] for col in data_filtered.columns if col[1] == 'Adj Close' and data_filtered[col].loc[target_start_date:].notnull().all()]
combined_data = data_filtered[valid_stocks].loc[target_start_date:]
from sklearn.preprocessing import LabelEncoder

# Flatten and encode tickers, filtered by `target_start_date`
def flatten_and_encode_tickers(data_filtered, valid_stocks, target_start_date):
    # Filter data for valid stocks and the target start date
    filtered_data = data_filtered[valid_stocks].loc[target_start_date:]

    # Initialize a list to store reshaped data
    reshaped_data = []

    # Iterate over tickers in valid_stocks
    for ticker in valid_stocks:
        # Extract data for the current ticker
        ticker_data = filtered_data[ticker].copy()
        # Add the ticker as a new column
        ticker_data['Ticker'] = ticker
        # Append to the list
        reshaped_data.append(ticker_data.reset_index())

    # Concatenate all ticker DataFrames into one
    flattened_data = pd.concat(reshaped_data, axis=0).reset_index(drop=True)

    # Sort by date
    flattened_data_sorted = flattened_data.sort_values(by='Date').reset_index(drop=True)

    # Encode the tickers
    label_encoder = LabelEncoder()
    flattened_data_sorted['Ticker_Encoded'] = label_encoder.fit_transform(flattened_data_sorted['Ticker'])

    return flattened_data_sorted, label_encoder


# Apply the flatten and encoding function
combined_data_flattened, label_encoder = flatten_and_encode_tickers(data_filtered, valid_stocks, target_start_date)


combined_data_flattened

2020-01-01 00:00:00


,Date,Adj Close,Volume,High_Low_Diff,Open_Close_Diff,Adj_Close_5d_Rolling,Adj_Close_10d_Rolling,Adj_Close_20d_Rolling,Volatility,Adj_Close_Up,RSI,MACD,Returns,Ticker,Ticker_Encoded
0,2020-01-01,209.965805,311952.0,13.596298,-2.719208,NaN,NaN,NaN,NaN,0.0,NaN,0.000000,NaN,ABRA.TA,0
1,2020-01-01,254.300003,62322.0,7.500000,-9.300003,NaN,NaN,NaN,NaN,0.0,NaN,0.000000,NaN,GIX.TA,125
2,2020-01-01,50.000000,20000.0,0.000000,0.000000,NaN,NaN,NaN,NaN,0.0,NaN,0.000000,NaN,ACCL.TA,1
3,2020-01-01,1349.000000,150.0,0.000000,-47.000000,NaN,NaN,NaN,NaN,0.0,NaN,0.000000,NaN,CRMT.TA,76
4,2020-01-01,1672.185669,283689.0,30.000000,-12.000000,NaN,NaN,NaN,NaN,0.0,NaN,0.000000,NaN,RIT1.TA,261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405470,2024-12-24,382.200012,6123.0,8.399994,-4.200012,378.820007,382.370001,382.394998,4.045856,1.0,45.714315,-2.848936,0.011111,HMAM.TA,145
405471,2024-12-24,1332.000000,684.0,20.000000,8.000000,1339.000000,1343.900000,1341.000000,7.279347,0.0,39.534884,4.013176,-0.005970,MSLA.TA,204
405472,2024-12-24,3008.000000,262925.0,64.000000,27.000000,3036.000000,3052.200000,3088.075879,52.393384,0.0,43.514644,9.319744,-0.008896,ALHE.TA,9
405473,2024-12-24,8458.000000,2650.0,70.000000,16.000000,8452.800000,8437.500000,8012.100000,78.896909,0.0,74.817340,418.338800,-0.001888,LVPR.TA,185


In [ ]:
target_data = combined_data_flattened[combined_data_flattened['Ticker'] == 'ABRA.TA']
non_target_data = combined_data_flattened[combined_data_flattened['Ticker'] != 'ABRA.TA']
target_features = target_data.drop(['Ticker', 'Adj Close'], axis=1)
non_target_features_pivoted = non_target_data.pivot(
    index='Date',
    columns='Ticker',
    values=[
        'Adj Close', 'Volume', 'High_Low_Diff', 'Open_Close_Diff',
        'Adj_Close_5d_Rolling', 'Adj_Close_10d_Rolling', 'Adj_Close_20d_Rolling',
        'Volatility', 'Adj_Close_Up', 'RSI', 'MACD', 'Returns'
    ]
)

# Flatten MultiIndex columns created by pivoting
non_target_features_pivoted.columns = ['{}_{}'.format(metric, ticker) for metric, ticker in non_target_features_pivoted.columns]

# Merge target features and non-target features on the Date
features = pd.concat([target_features.set_index('Date'), non_target_features_pivoted], axis=1).reset_index()
# sort features by date
features = features.sort_values(by='Date')
#drop date column
features = features.drop(['Date'], axis=1).fillna(method='ffill').fillna(method='bfill')
target = target_data['Adj Close']


<ipython-input-15-d728af4932ae>:22: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  features = features.drop(['Date'], axis=1).fillna(method='ffill').fillna(method='bfill')


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from scipy.optimize import minimize
from sklearn.preprocessing import LabelEncoder

# Input parameters
tickers = valid_tickers  # Add all tickers here
num_days_to_predict = 30  # Forecast days

# Initialize DataFrames to store results
# Initialize DataFrames to store results
actual_vs_pred_df = pd.DataFrame(columns=['Date', 'Ticker', 'Actual', 'Predicted'])
forecast_df = pd.DataFrame(columns=['Date', 'Ticker', 'Forecast', 'Days_Ahead'])
metrics_df = pd.DataFrame(columns=['Ticker', 'MSE', 'MAE','MAPE', 'R2'])

# lstm_better_df_path = '/content/drive/Shareddrives/capstone project-stock market liquidity/LSTM Results/lstm_better_df.csv'
# lstm_better_df = pd.read_csv(lstm_better_df_path)

# # Extract the tickers to process LSTM only for those
# tickers_to_process = lstm_better_df['Ticker'].unique()

# # Filter tickers based on this list
# tickers = [ticker for ticker in tickers if ticker in tickers_to_process]

valid_tickers = []
for ticker in tickers:
    ticker_data = data_filtered.xs(ticker, axis=1, level=0)['Adj Close']  # Extract 'Adj Close' for the ticker
    if ticker_data.max() >= 50:  # Check if the max value is >= 50
        valid_tickers.append(ticker)

# Update tickers list
tickers = valid_tickers

# Custom loss function to include volatility
import tensorflow.keras.backend as K
def mape(y_true, y_pred):
    return K.mean(K.abs((y_true - y_pred) / y_true)) * 100

# Define SMAPE
def smape(y_true, y_pred):
    denominator = K.abs(y_true) + K.abs(y_pred)
    return K.mean(2 * K.abs(y_true - y_pred) / denominator) * 100
def volatility_loss(y_true, y_pred):
    mse = K.mean(K.square(y_true - y_pred))
    diff_true = K.abs(y_true[1:] - y_true[:-1])
    diff_pred = K.abs(y_pred[1:] - y_pred[:-1])
    volatility_penalty = K.mean(K.square(diff_true - diff_pred))
    direction_error = K.mean(K.square(K.sign(y_true[1:] - y_true[:-1]) - K.sign(y_pred[1:] - y_pred[:-1])))
    return mse + 0.1 * volatility_penalty+0.1*direction_error+0.1*smape(y_true, y_pred)

# Function to create LSTM model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(LSTM(50, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss=volatility_loss)
    return model


# Flatten and encode tickers, filtered by `target_start_date`
def flatten_and_encode_tickers(data_filtered, valid_stocks, target_start_date):
    # Filter data for valid stocks and the target start date
    filtered_data = data_filtered[valid_stocks].loc[target_start_date:]

    # Initialize a list to store reshaped data
    reshaped_data = []

    # Iterate over tickers in valid_stocks
    for ticker in valid_stocks:
        # Extract data for the current ticker
        ticker_data = filtered_data[ticker].copy()
        # Add the ticker as a new column
        ticker_data['Ticker'] = ticker
        # Append to the list
        reshaped_data.append(ticker_data.reset_index())

    # Concatenate all ticker DataFrames into one
    flattened_data = pd.concat(reshaped_data, axis=0).reset_index(drop=True)

    # Sort by date
    flattened_data_sorted = flattened_data.sort_values(by='Date').reset_index(drop=True)

    # Encode the tickers
    label_encoder = LabelEncoder()
    flattened_data_sorted['Ticker_Encoded'] = label_encoder.fit_transform(flattened_data_sorted['Ticker'])

    return flattened_data_sorted, label_encoder




# Function to train and forecast for each stock
def process_stock(ticker, data_filtered):
    global actual_vs_pred_df, forecast_df, metrics_df

    print(f"Processing ticker: {ticker}")

    # Filter data for the target stock
    target_data = data_filtered[ticker].dropna(subset=['Adj Close'])
    target_start_date = target_data.index.min()
    print(f"Target start date for {ticker}: {target_start_date}")

    # Filter valid stocks based on overlapping data from `target_start_date`
    valid_stocks = [
        col[0] for col in data_filtered.columns if col[1] == 'Adj Close' and
        data_filtered[col].loc[target_start_date:].notnull().all()
    ]

    # Apply the updated function to flatten and encode tickers
    combined_data_flattened, label_encoder = flatten_and_encode_tickers(data_filtered, valid_stocks, target_start_date)
    combined_data_flattened = combined_data_flattened.fillna(method='ffill').fillna(method='bfill')
    if target_data.empty:
        print(f"No valid data for {ticker}. Skipping this stock.")
        return

    # Extract features and target
    target_data = combined_data_flattened[combined_data_flattened['Ticker'] == ticker]
    # Extract correlation-based ticker list for this ticker
    correlated_tickers = list(set(
        top_corr_values.get(ticker, []) + top_corr_trends.get(ticker, [])
    ))

    # Include the target ticker in the correlated tickers list
    correlated_tickers.append(ticker)
    print(f"Correlated tickers for {ticker}, which will be used in the model:")
    print(correlated_tickers)
    # Filter non-target data based on correlated tickers
    non_target_data = combined_data_flattened[combined_data_flattened['Ticker'].isin(correlated_tickers)]
    target_features = target_data.drop(['Ticker', 'Adj Close'], axis=1)
    non_target_features_pivoted = non_target_data.pivot(
        index='Date',
        columns='Ticker',
        values=[
            'Adj Close', 'Volume', 'High_Low_Diff', 'Open_Close_Diff',
            'Adj_Close_5d_Rolling', 'Adj_Close_10d_Rolling', 'Adj_Close_20d_Rolling',
            'Volatility', 'Adj_Close_Up', 'RSI', 'MACD', 'Returns'
        ]
    )

    # Flatten MultiIndex columns created by pivoting
    non_target_features_pivoted.columns = ['{}_{}'.format(metric, ticker) for metric, ticker in non_target_features_pivoted.columns]

    # Merge target features and non-target features on the Date
    features = pd.concat([target_features.set_index('Date'), non_target_features_pivoted], axis=1).reset_index()
    # sort features by date
    features = features.sort_values(by='Date')
    #drop date column
    features = features.drop(['Date'], axis=1).fillna(method='ffill').fillna(method='bfill')
    target = target_data['Adj Close']

    # Scaling
    scaler_X, scaler_y = RobustScaler(), RobustScaler()
    scaler_X, scaler_y = RobustScaler(), RobustScaler()
    try:
        X_scaled = scaler_X.fit_transform(features)
        y_scaled = scaler_y.fit_transform(target.values.reshape(-1, 1))
    except ValueError as e:
        print(f"Scaling error for {ticker}: {e}")
        return

    # Reshape for LSTM
    X_scaled = X_scaled.reshape(X_scaled.shape[0], 1, X_scaled.shape[1])

    # Split into train and test sets
    split_index = int(0.9 * len(X_scaled))
    X_train, X_test = X_scaled[:split_index], X_scaled[split_index:]
    y_train, y_test = y_scaled[:split_index], y_scaled[split_index:]
    train_dates, test_dates = target_data['Date'].iloc[:split_index], target_data['Date'].iloc[split_index:]

    # Train LSTM model
    model = create_lstm_model(input_shape=(X_train.shape[1], X_train.shape[2]))
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

    # Predict on test set
    y_pred = model.predict(X_test)
    y_pred_inverse = scaler_y.inverse_transform(y_pred)
    y_test_inverse = scaler_y.inverse_transform(y_test)
    # Evaluate performance
    mse = mean_squared_error(y_test_inverse, y_pred_inverse)
    mae = mean_absolute_error(y_test_inverse, y_pred_inverse)
    mape = mean_absolute_percentage_error(y_test_inverse, y_pred_inverse)
    r2 = r2_score(y_test_inverse, y_pred_inverse)

    print(f"Ticker: {ticker}, MSE: {mse}, MAE: {mae}, MAPE: {mape}, R2: {r2}")

    # Append metrics to DataFrame
    metrics_df.loc[len(metrics_df)] = [ticker, mse, mae, mape, r2]

    # Save actual vs predicted values
    temp_actual_pred = pd.DataFrame({
        'Date': test_dates,
        'Ticker': ticker,
        'Actual': y_test_inverse.flatten(),
        'Predicted': y_pred_inverse.flatten()
    })
    actual_vs_pred_df = pd.concat([actual_vs_pred_df, temp_actual_pred], ignore_index=True)

    # Forecast future values
    last_X = X_scaled[-1]
    future_predictions = []
    for _ in range(num_days_to_predict):
        future_pred = model.predict(last_X.reshape(1, 1, X_scaled.shape[2]))
        noise = np.random.normal(0, 0.05, future_pred.shape)  # Add noise for volatility
        future_pred_with_noise = future_pred + noise
        future_predictions.append(future_pred_with_noise[0, 0])
        last_X = np.roll(last_X, -1)
        last_X[-1] = future_pred_with_noise

    # Inverse scale forecast
    future_predictions_inverse = scaler_y.inverse_transform(np.array(future_predictions).reshape(-1, 1))

    # Save forecast
    future_dates = pd.date_range(target_data['Date'].iloc[-1] + pd.Timedelta(days=1), periods=num_days_to_predict)
    temp_forecast = pd.DataFrame({
        'Date': future_dates,
        'Ticker': ticker,
        'Forecast': future_predictions_inverse.flatten()
    })
    forecast_df = pd.concat([forecast_df, temp_forecast], ignore_index=True)

    # Visualization
    # plt.figure(figsize=(14, 8))
    # plt.plot(train_dates, scaler_y.inverse_transform(y_train), color='green', label='Train Actual')
    # plt.plot(test_dates, y_test_inverse, color='blue', label='Test Actual')
    # plt.plot(test_dates, y_pred_inverse, color='red', label='Test Predicted')
    # plt.plot(temp_forecast['Date'], temp_forecast['Forecast'], color='orange', label='Forecasted')
    # plt.title(f'{ticker} - Actual, Predicted, and Forecasted')
    # plt.xlabel('Date')
    # plt.ylabel('Adj Close')
    # plt.legend()
    # plt.grid(True)
    # plt.show()




# Process each stock
for ticker in tickers[200:]:
    process_stock(ticker, data_filtered)

# Save results
actual_vs_pred_df.to_csv('actual_vs_pred_from_200.csv', index=False)
forecast_df.to_csv('forecast_from_200.csv', index=False)
metrics_df.to_csv('metrics_from_200.csv', index=False)

In [ ]:
actual_vs_pred_df.to_csv('actual_vs_pred.csv', index=False)
forecast_df.to_csv('forecast.csv', index=False)
metrics_df.to_csv('metrics.csv', index=False)

In [ ]:
feature_data

NameError: name 'feature_data' is not defined

# GRU:

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, GRU
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from scipy.optimize import minimize

# Input parameters
tickers = valid_tickers  # Add all tickers here
num_days_to_predict = 30  # Forecast days

# Initialize DataFrames to store results
# Initialize DataFrames to store results
actual_vs_pred_df = pd.DataFrame(columns=['Date', 'Ticker', 'Actual', 'Predicted'])
forecast_df = pd.DataFrame(columns=['Date', 'Ticker', 'Forecast', 'Days_Ahead'])
metrics_df = pd.DataFrame(columns=['Ticker', 'MSE', 'MAE','MAPE', 'R2'])

# gru_better_df_path = '/content/drive/Shareddrives/capstone project-stock market liquidity/GRU results/gru_better_df.csv'
# gru_better_df = pd.read_csv(gru_better_df_path)

# # Extract the tickers to process gru only for those
# tickers_to_process = gru_better_df['Ticker'].unique()

# # Filter tickers based on this list
# tickers = [ticker for ticker in tickers if ticker in tickers_to_process]

valid_tickers = []
for ticker in tickers:
    ticker_data = data_filtered.xs(ticker, axis=1, level=0)['Adj Close']  # Extract 'Adj Close' for the ticker
    if ticker_data.max() >= 50:  # Check if the max value is >= 50
        valid_tickers.append(ticker)

# Update tickers list
tickers = valid_tickers

# Custom loss function to include volatility
import tensorflow.keras.backend as K
def mape(y_true, y_pred):
    return K.mean(K.abs((y_true - y_pred) / y_true)) * 100

# Define SMAPE
def smape(y_true, y_pred):
    denominator = K.abs(y_true) + K.abs(y_pred)
    return K.mean(2 * K.abs(y_true - y_pred) / denominator) * 100
def volatility_loss(y_true, y_pred):
    mse = K.mean(K.square(y_true - y_pred))
    diff_true = K.abs(y_true[1:] - y_true[:-1])
    diff_pred = K.abs(y_pred[1:] - y_pred[:-1])
    volatility_penalty = K.mean(K.square(diff_true - diff_pred))
    direction_error = K.mean(K.square(K.sign(y_true[1:] - y_true[:-1]) - K.sign(y_pred[1:] - y_pred[:-1])))
    return mse + 0.1 * volatility_penalty+0.1*direction_error+0.1*smape(y_true, y_pred)

# Function to create GRU model
def create_gru_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(GRU(50, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(GRU(50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss=volatility_loss)
    return model

# Function to train and forecast for each stock
def process_stock(ticker, data_filtered):
    global actual_vs_pred_df, forecast_df, metrics_df

    print(f"Processing ticker: {ticker}")

    # Extract data for the ticker
    ticker_data = data_filtered.xs(ticker, axis=1, level=0)
    ticker_data['Volatility'] = ticker_data['Adj Close'].rolling(window=10).std()
    ticker_data['Returns'] = ticker_data['Adj Close'].pct_change(fill_method=None)
    ticker_data = ticker_data.dropna()

    # Features and target
    X = ticker_data.drop('Adj Close', axis=1)
    y = ticker_data['Adj Close']
    dates = ticker_data.index

    # Scaling
    scaler_X, scaler_y = RobustScaler(), RobustScaler()
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))
    X_scaled = X_scaled.reshape(X_scaled.shape[0], 1, X_scaled.shape[1])

    # Split data
    split_index = int(0.9 * len(X_scaled))
    X_train, X_test = X_scaled[:split_index], X_scaled[split_index:]
    y_train, y_test = y_scaled[:split_index], y_scaled[split_index:]
    train_dates, test_dates = dates[:split_index], dates[split_index:]

    # Build GRU model
    model = create_gru_model(input_shape=(X_train.shape[1], X_train.shape[2]))
    model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=0)

    # Predict
    y_pred = model.predict(X_test)
    def compute_correction_factor(y_actual, y_pred, window_size=30):
      correction_factors = []
      for start in range(0, len(y_actual), window_size):
          end = start + window_size
          actual_slice = y_actual[start:end]
          pred_slice = y_pred[start:end]
          if len(actual_slice) > 0 and np.mean(pred_slice) > 0:
              correction_factors.append(np.mean(actual_slice) / np.mean(pred_slice))
          else:
              correction_factors.append(1)  # Default to no correction if invalid
      return correction_factors
    def apply_correction(y_pred, correction_factors, window_size=30):
      corrected_predictions = []
      for i, factor in enumerate(correction_factors):
          start = i * window_size
          end = start + window_size
          corrected_predictions.extend(y_pred[start:end] * factor)
      return np.array(corrected_predictions)


    y_pred_inverse = scaler_y.inverse_transform(y_pred)
    y_test_inverse = scaler_y.inverse_transform(y_test)
    if np.any(np.isnan(y_test_inverse)) or np.any(np.isnan(y_pred_inverse)):
        print(f"NaN values detected for ticker: {ticker}")
        # Remove NaN values
        y_test_inverse = y_test_inverse[~np.isnan(y_test_inverse)]
        y_pred_inverse = y_pred_inverse[~np.isnan(y_pred_inverse)]

    # Ensure both arrays have the same length
    if len(y_test_inverse) == 0 or len(y_pred_inverse) == 0:
        print(f"No valid predictions for ticker: {ticker}. Skipping metric calculation.")
        return

    adjustment = y_test_inverse[0] - y_pred_inverse[0]

    # Apply the adjustment to all predicted values
    y_pred_inverse_adjusted = y_pred_inverse + adjustment

    # Ensure both arrays have the same length after adjustment
    if len(y_test_inverse) != len(y_pred_inverse_adjusted):
        min_length = min(len(y_test_inverse), len(y_pred_inverse_adjusted))
        y_test_inverse = y_test_inverse[:min_length]
        y_pred_inverse_adjusted = y_pred_inverse_adjusted[:min_length]

    # Calculate metrics with adjusted predictions
    mse = mean_squared_error(y_test_inverse, y_pred_inverse_adjusted)
    mae = mean_absolute_error(y_test_inverse, y_pred_inverse_adjusted)
    mape = mean_absolute_percentage_error(y_test_inverse, y_pred_inverse_adjusted)
    r2 = r2_score(y_test_inverse, y_pred_inverse_adjusted)

    print(f"Ticker: {ticker}, MSE: {mse}, MAE: {mae}, MAPE: {mape}, R2: {r2}")

    # Append metrics to DataFrame
    metrics_df.loc[len(metrics_df)] = [ticker, mse, mae, mape, r2]

    # Append actual vs adjusted predicted values
    temp_actual_pred = pd.DataFrame({
        'Date': test_dates,
        'Ticker': ticker,
        'Actual': y_test_inverse.flatten(),
        'Predicted': y_pred_inverse_adjusted.flatten()
    })
    actual_vs_pred_df = pd.concat([actual_vs_pred_df, temp_actual_pred], ignore_index=True)


    # Forecast next n days
    last_X = X_scaled[-1]
    future_predictions = []
    for _ in range(num_days_to_predict):
        future_pred = model.predict(last_X.reshape(1, 1, X_scaled.shape[2]))
        noise = np.random.normal(0, 0.05, future_pred.shape)  # Add noise for volatility
        future_pred_with_noise = future_pred + noise
        future_predictions.append(future_pred_with_noise[0, 0])
        last_X = np.roll(last_X, -1)
        last_X[-1] = future_pred_with_noise

    # Inverse scale forecast
    future_predictions_inverse = scaler_y.inverse_transform(np.array(future_predictions).reshape(-1, 1))

    # Append forecast to DataFrame
    future_dates = pd.date_range(dates[-1] + pd.Timedelta(days=1), periods=num_days_to_predict)
    temp_forecast = pd.DataFrame({
        'Date': future_dates,
        'Ticker': ticker,
        'Forecast': future_predictions_inverse.flatten(),
        'Days_Ahead': np.arange(1, num_days_to_predict + 1)
    })
    forecast_df = pd.concat([forecast_df, temp_forecast], ignore_index=True)

    # Plot results
    plt.figure(figsize=(14, 8))
    plt.plot(train_dates, scaler_y.inverse_transform(y_train), color='green', label='Actual Adj Close (Train)')
    plt.plot(test_dates, y_test_inverse, color='blue', label='Actual Adj Close (Test)')
    plt.plot(test_dates, y_pred_inverse_adjusted, color='red', label='Predicted Adj Close (Test)')
    plt.plot(temp_forecast['Date'], temp_forecast['Forecast'], color='orange', label='Forecasted Prices')

    plt.title('Stock Prices: Actual, Predicted, and Forecasted')
    plt.xlabel('Date')
    plt.ylabel('Adj Close')
    plt.legend()
    plt.show()

# Load data
start_date = "2020-01-01"
end_date = pd.Timestamp.today().strftime('%Y-%m-%d')
data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')

# Process each stock
for ticker in tickers[306:]:
    process_stock(ticker, data)

# Save results
actual_vs_pred_df.to_csv('actual_vs_pred_gru.csv', index=False)
forecast_df.to_csv('forecast.csv_gru', index=False)
metrics_df.to_csv('metrics.csv_gru', index=False)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
forecast_df.to_csv('forecast_gru.csv', index=False)
metrics_df.to_csv('metrics_gru.csv', index=False)

# Indices LSTM:

In [ ]:
pip install yfinance

In [ ]:
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
from google.colab import drive
import pandas as pd
import yfinance as yf
from datetime import datetime

drive.mount('/content/drive')

# File paths
symbols_file_path = '/content/drive/Shareddrives/capstone project-stock market liquidity/TA_indices_symbols.csv'
historical_file_path = '/content/drive/Shareddrives/capstone project-stock market liquidity/index raw data.csv'

# Load CSV files
symbols_df = pd.read_csv(symbols_file_path)
historical_df = pd.read_csv(historical_file_path)

# Display the first few rows to ensure files are loaded correctly
print("Symbols DataFrame:")
print(symbols_df.head())

print("\nHistorical Data DataFrame:")
print(historical_df.head())

# Merge the two dataframes using 'Index Code' as the key
merged_df = pd.merge(historical_df, symbols_df, on='Index Code', how='inner')

# Extract valid tickers
valid_tickers = merged_df['Index Name_y'].unique().tolist()
print("\nValid Tickers:", valid_tickers)

# Feature engineering
# Calculate High-Low and Open-Close differences
merged_df['High_Low_Diff'] = merged_df['High'] - merged_df['Low']
merged_df['Open_Close_Diff'] = merged_df['Index Opening Price'] - merged_df['Closing Index Price']

# Rolling averages
merged_df['Close_5d_Rolling'] = merged_df['Closing Index Price'].rolling(window=5).mean()
merged_df['Close_10d_Rolling'] = merged_df['Closing Index Price'].rolling(window=10).mean()
merged_df['Close_20d_Rolling'] = merged_df['Closing Index Price'].rolling(window=20).mean()

# Volatility
merged_df['Volatility'] = merged_df['Closing Index Price'].rolling(window=20).std()

# Add Binary Feature: Is Closing Index Price up (1) or down (0) compared to the previous day
merged_df['Close_Up'] = (merged_df['Closing Index Price'].diff() > 0).astype(int)

# RSI Calculation
def calculate_rsi(prices, period=14):
    prices = pd.Series(prices).astype(float).dropna()
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

# MACD Calculation
def calculate_macd(prices, fast=12, slow=26):
    prices = pd.Series(prices).astype(float).dropna()
    exp1 = prices.ewm(span=fast, adjust=False).mean()
    exp2 = prices.ewm(span=slow, adjust=False).mean()
    return exp1 - exp2

# Calculate RSI and MACD
merged_df['RSI'] = calculate_rsi(merged_df['Closing Index Price'])
merged_df['MACD'] = calculate_macd(merged_df['Closing Index Price'])

# Print final DataFrame with new features
print("\nFinal DataFrame with Features:")
print(merged_df.head())

# Save the processed data to a CSV file
processed_file_path = '/content/drive/Shareddrives/capstone project-stock market liquidity/processed_data.csv'
merged_df.to_csv(processed_file_path, index=False)
print(f"\nProcessed data saved to: {processed_file_path}")


In [ ]:
merged_df

In [ ]:
if '2024-04-07' in data_filtered.index:
    # Get the index position of '2024-04-07'
    idx = data_filtered.index.get_loc('2024-04-07')

    # Replace each column's value for '2024-04-07' with its next row's value
    for col in data_filtered.columns:
        if idx + 1 < len(data_filtered):  # Ensure the next row exists
            next_value = data_filtered.iloc[idx + 1][col]
            data_filtered.loc['2024-04-07', col] = next_value

In [ ]:
data_filtered

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from scipy.optimize import minimize

# Input parameters
tickers = valid_tickers  # Add all tickers here
num_days_to_predict = 30  # Forecast days

# Initialize DataFrames to store results
actual_vs_pred_df = pd.DataFrame(columns=['Date', 'Ticker', 'Actual', 'Predicted'])
forecast_df = pd.DataFrame(columns=['Date', 'Ticker', 'Forecast', 'Days_Ahead'])
metrics_df = pd.DataFrame(columns=['Ticker', 'MSE', 'MAE', 'R2'])

# Function to create LSTM model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))  # Use Input layer to avoid warnings
    model.add(LSTM(50, return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dropout(0.1))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Function to train the model using LBFGS
def train_with_lbfgs(model, X_train, Y_train):
    # Flatten model weights into a single vector
    initial_weights = model.get_weights()
    flattened_weights = np.concatenate([w.flatten() for w in initial_weights])
    weight_shapes = [w.shape for w in initial_weights]

    # Loss function for LBFGS
    def loss_fn(weights_flattened):
        # Unflatten weights and set them back to the model
        unflattened_weights = []
        offset = 0
        for shape in weight_shapes:
            size = np.prod(shape)
            unflattened_weights.append(weights_flattened[offset:offset + size].reshape(shape))
            offset += size
        model.set_weights(unflattened_weights)

        # Compute the loss
        predictions = model(X_train, training=False)  # Use the model directly
        loss = tf.reduce_mean(tf.square(Y_train - predictions))  # Use TensorFlow operations
        return loss  # Return as a TensorFlow tensor

    # Gradient function for LBFGS
    def gradient_fn(weights_flattened):
        with tf.GradientTape() as tape:
            # Unflatten weights and set them back to the model
            unflattened_weights = []
            offset = 0
            for shape in weight_shapes:
                size = np.prod(shape)
                unflattened_weights.append(weights_flattened[offset:offset + size].reshape(shape))
                offset += size
            model.set_weights(unflattened_weights)

            # Compute the loss
            predictions = model(X_train, training=False)
            loss = tf.reduce_mean(tf.square(Y_train - predictions))

        # Compute gradients
        gradients = tape.gradient(loss, model.trainable_variables)
        flattened_gradients = np.concatenate([g.numpy().flatten() for g in gradients])
        return flattened_gradients

    # Perform LBFGS optimization
    result = minimize(
        fun=loss_fn,
        x0=flattened_weights,
        jac=gradient_fn,
        method='L-BFGS-B',
        options={'maxiter': 100, 'disp': True}
    )

    # Set the final optimized weights back to the model
    optimized_weights = []
    offset = 0
    for shape in weight_shapes:
        size = np.prod(shape)
        optimized_weights.append(result.x[offset:offset + size].reshape(shape))
        offset += size
    model.set_weights(optimized_weights)

# Function to train and forecast for each stock
# Function to train and forecast for each stock
def process_stock(ticker, data_filtered):
    global actual_vs_pred_df, forecast_df, metrics_df

    print(f"Processing ticker: {ticker}")

    # Extract data for the ticker
    ticker_data = merged_df[merged_df['Index Name_y'] == ticker].copy()

    # Convert dates to index
    ticker_data['Date'] = pd.to_datetime(ticker_data['Date'])
    # sort ticker_data by date ascending
    ticker_data.sort_values(by='Date', ascending=True, inplace=True)
    ticker_data.set_index('Date', inplace=True)

    # Keep only numeric columns
    ticker_data = ticker_data.select_dtypes(include=[np.number])

    # Check if there's sufficient data after filtering
    if ticker_data.empty or len(ticker_data) < 10:
        print(f"Insufficient data for ticker: {ticker}")
        return

    # Separate features (X) and target (y)
    y = ticker_data['Closing Index Price']
    X = ticker_data.drop('Closing Index Price', axis=1)

    # Drop NaNs
    combined = pd.concat([X, y], axis=1).dropna()
    y = combined['Closing Index Price'].values
    dates = combined.index

    # Scaling
    scaler_X, scaler_y = MinMaxScaler(), MinMaxScaler()
    X_scaled = scaler_X.fit_transform(combined.drop('Closing Index Price', axis=1).values)
    y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))
    X_scaled = X_scaled.reshape(X_scaled.shape[0], 1, X_scaled.shape[1])

    # Split data
    split_index = int(0.9 * len(X_scaled))
    X_train, X_test = X_scaled[:split_index], X_scaled[split_index:]
    y_train, y_test = y_scaled[:split_index], y_scaled[split_index:]
    train_dates, test_dates = dates[:split_index], dates[split_index:]

    # Build LSTM model
    model = create_lstm_model(input_shape=(X_train.shape[1], X_train.shape[2]))

    # Train the model using LBFGS
    train_with_lbfgs(model, X_train, y_train)

    # Predict
    y_pred = model.predict(X_test)
    y_pred_inverse = scaler_y.inverse_transform(y_pred)
    y_test_inverse = scaler_y.inverse_transform(y_test)

    # Calculate metrics
    mse = mean_squared_error(y_test_inverse, y_pred_inverse)
    mae = mean_absolute_error(y_test_inverse, y_pred_inverse)
    r2 = r2_score(y_test_inverse, y_pred_inverse)
    print(f"Ticker: {ticker}, MSE: {mse}, MAE: {mae}, R2: {r2}")

    # Append metrics to DataFrame
    metrics_df.loc[len(metrics_df)] = [ticker, mse, mae, r2]

    # Append actual vs predicted values
    temp_actual_pred = pd.DataFrame({
        'Date': test_dates,
        'Ticker': ticker,
        'Actual': y_test_inverse.flatten(),
        'Predicted': y_pred_inverse.flatten()
    })
    actual_vs_pred_df = pd.concat([actual_vs_pred_df, temp_actual_pred], ignore_index=True)

    # Forecast next n days
    last_X = X_scaled[-1]
    future_predictions = []
    for _ in range(num_days_to_predict):
        future_pred = model.predict(last_X.reshape(1, 1, X_scaled.shape[2]))
        future_predictions.append(future_pred[0, 0])
        last_X = np.roll(last_X, -1)
        last_X[-1] = future_pred

    # Inverse scale forecast
    future_predictions_inverse = scaler_y.inverse_transform(np.array(future_predictions).reshape(-1, 1))

    # Append forecast to DataFrame
    future_dates = pd.date_range(dates[-1] + pd.Timedelta(days=1), periods=num_days_to_predict)
    temp_forecast = pd.DataFrame({
        'Date': future_dates,
        'Ticker': ticker,
        'Forecast': future_predictions_inverse.flatten(),
        'Days_Ahead': np.arange(1, num_days_to_predict + 1)
    })
    forecast_df = pd.concat([forecast_df, temp_forecast], ignore_index=True)

    # Plot properly aligned dates and predictions
    plt.figure(figsize=(12, 6))
    plt.plot(train_dates, scaler_y.inverse_transform(y_train), color='green', label='Closing Index Price (Train)')
    plt.plot(test_dates, y_test_inverse, color='blue', label='Actual Closing Index Price (Test)')
    plt.plot(test_dates, y_pred_inverse, color='red', label='Predicted Closing Index Price (Test)')

    plt.title('Actual vs Predicted Closing Index Price')
    plt.xlabel('Date')
    plt.ylabel('Adj Closing Index Price')
    plt.legend()
    plt.show()


# Load data
start_date = "2020-01-01"
end_date = pd.Timestamp.today().strftime('%Y-%m-%d')
data = merged_df

# Flatten the MultiIndex columns for processing
data_filtered = data.copy()

# Process each stock
for ticker in tickers:
    process_stock(ticker, data_filtered)

# Save DataFrames
actual_vs_pred_df.to_csv('actual_vs_pred_indices.csv', index=False)
forecast_df.to_csv('forecast_indices.csv', index=False)
metrics_df.columns = ['Ticker', 'MSE', 'MAE', 'R2']
metrics_df.to_csv('metrics_indices.csv', index=False)

## GRU:

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, Input
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from scipy.optimize import minimize

# Input parameters
tickers = valid_tickers  # Add all tickers here
num_days_to_predict = 30  # Forecast days

# Initialize DataFrames to store results
actual_vs_pred_df = pd.DataFrame(columns=['Date', 'Ticker', 'Actual', 'Predicted'])
forecast_df = pd.DataFrame(columns=['Date', 'Ticker', 'Forecast', 'Days_Ahead'])
metrics_df = pd.DataFrame(columns=['Ticker', 'MSE', 'MAE', 'R2'])

# Function to create GRU model
def create_gru_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))  # Use Input layer to avoid warnings
    model.add(GRU(50, return_sequences=True))  # Replace LSTM with GRU
    model.add(Dropout(0.1))
    model.add(GRU(50, return_sequences=False))  # Replace LSTM with GRU
    model.add(Dropout(0.1))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


# Function to train the model using LBFGS
def train_with_lbfgs(model, X_train, Y_train):
    # Flatten model weights into a single vector
    initial_weights = model.get_weights()
    flattened_weights = np.concatenate([w.flatten() for w in initial_weights])
    weight_shapes = [w.shape for w in initial_weights]

    # Loss function for LBFGS
    def loss_fn(weights_flattened):
        # Unflatten weights and set them back to the model
        unflattened_weights = []
        offset = 0
        for shape in weight_shapes:
            size = np.prod(shape)
            unflattened_weights.append(weights_flattened[offset:offset + size].reshape(shape))
            offset += size
        model.set_weights(unflattened_weights)

        # Compute the loss
        predictions = model(X_train, training=False)  # Use the model directly
        loss = tf.reduce_mean(tf.square(Y_train - predictions))  # Use TensorFlow operations
        return loss  # Return as a TensorFlow tensor

    # Gradient function for LBFGS
    def gradient_fn(weights_flattened):
        with tf.GradientTape() as tape:
            # Unflatten weights and set them back to the model
            unflattened_weights = []
            offset = 0
            for shape in weight_shapes:
                size = np.prod(shape)
                unflattened_weights.append(weights_flattened[offset:offset + size].reshape(shape))
                offset += size
            model.set_weights(unflattened_weights)

            # Compute the loss
            predictions = model(X_train, training=False)
            loss = tf.reduce_mean(tf.square(Y_train - predictions))

        # Compute gradients
        gradients = tape.gradient(loss, model.trainable_variables)
        flattened_gradients = np.concatenate([g.numpy().flatten() for g in gradients])
        return flattened_gradients

    # Perform LBFGS optimization
    result = minimize(
        fun=loss_fn,
        x0=flattened_weights,
        jac=gradient_fn,
        method='L-BFGS-B',
        options={'maxiter': 100, 'disp': True}
    )

    # Set the final optimized weights back to the model
    optimized_weights = []
    offset = 0
    for shape in weight_shapes:
        size = np.prod(shape)
        optimized_weights.append(result.x[offset:offset + size].reshape(shape))
        offset += size
    model.set_weights(optimized_weights)

# Function to train and forecast for each stock
def process_stock(ticker, data_filtered):
    global actual_vs_pred_df, forecast_df, metrics_df

    print(f"Processing ticker: {ticker}")

    # Extract data for the ticker
    ticker_data = merged_df[merged_df['Index Name_y'] == ticker].copy()

    # Convert dates to index
    ticker_data['Date'] = pd.to_datetime(ticker_data['Date'])
    # sort ticker_data by date ascending
    ticker_data.sort_values(by='Date', ascending=True, inplace=True)
    ticker_data.set_index('Date', inplace=True)

    # Keep only numeric columns
    ticker_data = ticker_data.select_dtypes(include=[np.number])

    # Check if there's sufficient data after filtering
    if ticker_data.empty or len(ticker_data) < 10:
        print(f"Insufficient data for ticker: {ticker}")
        return

    # Separate features (X) and target (y)
    y = ticker_data['Closing Index Price']
    X = ticker_data.drop('Closing Index Price', axis=1)

    # Drop NaNs
    combined = pd.concat([X, y], axis=1).dropna()
    y = combined['Closing Index Price'].values
    dates = combined.index

    # Scaling
    scaler_X, scaler_y = MinMaxScaler(), MinMaxScaler()
    X_scaled = scaler_X.fit_transform(combined.drop('Closing Index Price', axis=1).values)
    y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))
    X_scaled = X_scaled.reshape(X_scaled.shape[0], 1, X_scaled.shape[1])

    # Split data
    split_index = int(0.9 * len(X_scaled))
    X_train, X_test = X_scaled[:split_index], X_scaled[split_index:]
    y_train, y_test = y_scaled[:split_index], y_scaled[split_index:]
    train_dates, test_dates = dates[:split_index], dates[split_index:]

    # Build LSTM model
    model = create_gru_model(input_shape=(X_train.shape[1], X_train.shape[2]))

    # Train the model using LBFGS
    train_with_lbfgs(model, X_train, y_train)

    # Predict
    y_pred = model.predict(X_test)
    y_pred_inverse = scaler_y.inverse_transform(y_pred)
    y_test_inverse = scaler_y.inverse_transform(y_test)

    # Calculate metrics
    mse = mean_squared_error(y_test_inverse, y_pred_inverse)
    mae = mean_absolute_error(y_test_inverse, y_pred_inverse)
    r2 = r2_score(y_test_inverse, y_pred_inverse)
    print(f"Ticker: {ticker}, MSE: {mse}, MAE: {mae}, R2: {r2}")

    # Append metrics to DataFrame
    metrics_df.loc[len(metrics_df)] = [ticker, mse, mae, r2]

    # Append actual vs predicted values
    temp_actual_pred = pd.DataFrame({
        'Date': test_dates,
        'Ticker': ticker,
        'Actual': y_test_inverse.flatten(),
        'Predicted': y_pred_inverse.flatten()
    })
    actual_vs_pred_df = pd.concat([actual_vs_pred_df, temp_actual_pred], ignore_index=True)

    # Forecast next n days
    last_X = X_scaled[-1]
    future_predictions = []
    for _ in range(num_days_to_predict):
        future_pred = model.predict(last_X.reshape(1, 1, X_scaled.shape[2]))
        future_predictions.append(future_pred[0, 0])
        last_X = np.roll(last_X, -1)
        last_X[-1] = future_pred

    # Inverse scale forecast
    future_predictions_inverse = scaler_y.inverse_transform(np.array(future_predictions).reshape(-1, 1))

    # Append forecast to DataFrame
    future_dates = pd.date_range(dates[-1] + pd.Timedelta(days=1), periods=num_days_to_predict)
    temp_forecast = pd.DataFrame({
        'Date': future_dates,
        'Ticker': ticker,
        'Forecast': future_predictions_inverse.flatten(),
        'Days_Ahead': np.arange(1, num_days_to_predict + 1)
    })
    forecast_df = pd.concat([forecast_df, temp_forecast], ignore_index=True)

    # Plot properly aligned dates and predictions
    plt.figure(figsize=(12, 6))
    plt.plot(train_dates, scaler_y.inverse_transform(y_train), color='green', label='Closing Index Price (Train)')
    plt.plot(test_dates, y_test_inverse, color='blue', label='Actual Closing Index Price (Test)')
    plt.plot(test_dates, y_pred_inverse, color='red', label='Predicted Closing Index Price (Test)')

    plt.title('Actual vs Predicted Closing Index Price')
    plt.xlabel('Date')
    plt.ylabel('Adj Closing Index Price')
    plt.legend()
    plt.show()


# Load data
start_date = "2020-01-01"
end_date = pd.Timestamp.today().strftime('%Y-%m-%d')
data = merged_df

# Flatten the MultiIndex columns for processing
data_filtered = data.copy()

# Process each stock
for ticker in tickers:
    process_stock(ticker, data_filtered)

# Save DataFrames
actual_vs_pred_df.to_csv('actual_vs_pred_indices_gru.csv', index=False)
forecast_df.to_csv('forecast_indices_gru.csv', index=False)
metrics_df.columns = ['Ticker', 'MSE', 'MAE', 'R2']
metrics_df.to_csv('metrics_indices_gru.csv', index=False)

In [ ]:
import matplotlib.dates as mdates

# Plot 2: 5-Day Forecast (with correct X-axis formatting)
plt.figure(figsize=(12, 6))

# Plot the 5-day future forecast SPY Adj Close
plt.plot(future_dates, future_predictions_inverse, color='purple', label='Forecasted SPY Adj Close (5 Days)')

# Formatting the x-axis to show only the date (without time)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator())

plt.title('SPY Adj Close: 5-Day Forecast')
plt.xlabel('Date')
plt.ylabel('SPY Adj Close')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)  # Rotate the x-axis labels for better readability

# ... קוד ליצירת הגרף ...

plt.grid(False)
plt.show()
plt.show()


In [ ]:
import shap

predict_fn = lambda x: model.predict(x.reshape((x.shape[0], 1, x.shape[1])))

explainer_shap = shap.SamplingExplainer(predict_fn, X_train.reshape(X_train.shape[0], X_train.shape[2]), n_jobs=-1)

X_explain_shap = X_test.reshape(X_test.shape[0], -1)  # Reshape to 2D

shap_values = explainer_shap.shap_values(X_explain_shap)

feature_names = [f'{ticker}_{feature}' for ticker in tickers for feature in
                 ['Adj Close', 'Volume', 'High_Low_Diff', 'Open_Close_Diff', 'Adj_Close_5d_Rolling',
                  'Adj_Close_10d_Rolling', 'Adj_Close_20d_Rolling']]

shap.summary_plot(shap_values, X_explain_shap, feature_names=feature_names)

In [ ]:
# Assuming X_test is your test data and model is your trained LSTM model
import lime
import lime.lime_tabular
import numpy as np

# Reshape X_test to be 2D for LIME
X_test_reshaped = X_test.reshape(X_test.shape[0], -1)

# Create the LIME explainer
explainer = lime.lime_tabular.LimeTabularExplainer(
    X_test_reshaped,
    mode='regression',
    feature_names=[f'{ticker}_{feature}' for ticker in tickers for feature in
                   ['Adj Close', 'Volume', 'High_Low_Diff', 'Open_Close_Diff', 'Adj_Close_5d_Rolling',
                    'Adj_Close_10d_Rolling', 'Adj_Close_20d_Rolling']],
    discretize_continuous=True
)

# Choose an instance to explain
instance_idx = 0
X_explain = X_test[instance_idx].reshape(1, -1)

# Define a prediction function for LIME that reshapes the input for the LSTM model
def predict_fn(x):
    x_reshaped = x.reshape(x.shape[0], 1, X_test.shape[2])
    return model.predict(x_reshaped)

# Generate the explanation
exp = explainer.explain_instance(X_explain.flatten(), predict_fn, num_features=10)

# Display the explanation
exp.show_in_notebook()

In [ ]:
model.summary()

In [ ]:
import shap

# פונקציה לניבוי שתתאים את מבנה הנתונים למודל LSTM
predict_fn = lambda x: model.predict(x.reshape((x.shape[0], 1, x.shape[1])))

# שימוש ב-KernelExplainer של SHAP עבור מודל ה-LSTM
# X_train צריך להיות במבנה המתאים עבור SHAP - מבנה דו-ממדי
explainer_shap = shap.KernelExplainer(predict_fn, X_train.reshape(X_train.shape[0], X_train.shape[2]))

# בוחרים דגימה להסבר מ-X_test
# X_explain_shap = X_test[0].reshape(1, -1)  # מישוט הדגימה עבור SHAP
X_explain_shap = X_test.reshape(X_test.shape[0], -1)  # Reshape to 2D

# קבלת ערכי SHAP עבור הדגימה
shap_values = explainer_shap.shap_values(X_explain_shap)

# הצגת גרף SHAP עבור הדגימה
shap.summary_plot(shap_values[0], X_explain_shap)


## What is better-lstm or gru?

In [ ]:
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the metric files
metric_lstm = pd.read_csv('/content/drive/Shareddrives/capstone project-stock market liquidity/LSTM Results/metrics.csv')
metrics_indices_lstm = pd.read_csv('/content/drive/Shareddrives/capstone project-stock market liquidity/LSTM Results/metrics_indices.csv')
metric_gru = pd.read_csv('/content/drive/Shareddrives/capstone project-stock market liquidity/GRU results/metrics_gru.csv')
metrics_indices_gru = pd.read_csv('/content/drive/Shareddrives/capstone project-stock market liquidity/GRU results/metrics_indices_gru.csv')

# Initialize counters and result storage
lstm_points = 0
gru_points = 0
results = []

# Function to compare metrics and store results
def compare_metrics_and_store(lstm_df, gru_df, key_column):
    global lstm_points, gru_points
    for _, row_lstm in lstm_df.iterrows():
        key = row_lstm[key_column]
        row_gru = gru_df[gru_df[key_column] == key]
        if not row_gru.empty:
            for metric in ['R2', 'MSE', 'MAE']:
                lstm_value = row_lstm[metric]
                gru_value = row_gru[metric].values[0]
                if metric == 'R2':  # Higher is better
                    if lstm_value > gru_value:
                        lstm_points += 1
                        better_model = 'LSTM'
                    elif gru_value > lstm_value:
                        gru_points += 1
                        better_model = 'GRU'
                    else:
                        better_model = 'Tie'
                else:  # Lower is better for MSE and MAE
                    if lstm_value < gru_value:
                        lstm_points += 1
                        better_model = 'LSTM'
                    elif gru_value < lstm_value:
                        gru_points += 1
                        better_model = 'GRU'
                    else:
                        better_model = 'Tie'
                results.append({
                    'Ticker': key,
                    'Metric': metric,
                    'LSTM_Value': lstm_value,
                    'GRU_Value': gru_value,
                    'Better_Model': better_model
                })

# Compare stock-level metrics
compare_metrics_and_store(metric_lstm, metric_gru, 'Ticker')

# Compare index-level metrics
compare_metrics_and_store(metrics_indices_lstm, metrics_indices_gru, 'Ticker')

# Print the overall results
print(f"LSTM points: {lstm_points}")
print(f"GRU points: {gru_points}")
if lstm_points > gru_points:
    print("LSTM performed better overall.")
elif gru_points > lstm_points:
    print("GRU performed better overall.")
else:
    print("Both models performed equally well.")

# Create DataFrame for detailed results
results_df = pd.DataFrame(results)

In [ ]:
results_df

In [ ]:
model_counts = results_df.groupby(['Ticker', 'Better_Model']).size().unstack(fill_value=0)

# Extract tickers where at least 2 metrics favor LSTM
tickers_better_lstm = model_counts[model_counts['LSTM'] >= 2].index.tolist()

# Extract tickers where at least 2 metrics favor GRU
tickers_better_gru = model_counts[model_counts['GRU'] >= 2].index.tolist()

# Create DataFrames for the results
lstm_better_df = pd.DataFrame({'Ticker': tickers_better_lstm, 'Better_Model': 'LSTM'})
gru_better_df = pd.DataFrame({'Ticker': tickers_better_gru, 'Better_Model': 'GRU'})

In [ ]:
lstm_better_df.to_csv('/content/drive/Shareddrives/capstone project-stock market liquidity/LSTM Results/lstm_better_df.csv', index=False)
gru_better_df.to_csv('/content/drive/Shareddrives/capstone project-stock market liquidity/GRU results/gru_better_df.csv', index=False)

In [ ]:
gru_better_df